In [90]:
from games_setup import *
import SBMLLint.common.constants as cn
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.som import SOM
from SBMLLint.games.mesgraph import MESGraph
import collections
import tesbml
import numpy as np
import pandas as pd

In [6]:
data_dir=cn.DATA_DIR
files = [f for f in os.listdir(data_dir) if f[-4:] == ".xml"]
paths = [os.path.join(data_dir, filename) for filename in files]

In [19]:
# statistics columns
LP_ERROR = "LP_Error"
MESGRAPH_ERROR = "MESGraph_Error"
TYPE_I = "Type_I"
TYPE_II = "Type_II"
result_columns = [LP_ERROR, MESGRAPH_ERROR, TYPE_I, TYPE_II]

In [20]:
results = pd.DataFrame(False, index=files, columns=result_columns)
results[:5]

,LP_Error,MESGraph_Error,Type_I,Type_II
BIOMD0000000199_url.xml,False,False,False,False
BIOMD0000000189_url.xml,False,False,False,False
BIOMD0000000387_url.xml,False,False,False,False
BIOMD0000000397_url.xml,False,False,False,False
BIOMD0000000413_url.xml,False,False,False,False


In [18]:
results.loc[files[0]][LP_ERROR]

False

In [ ]:
# Checking all graphs 
simple = SimpleSBML()
for file in files:
  try:
    simple.initialize(os.path.join(data_dir, file))
    m = MESGraph(simple)
    s = StoichiometryMatrix(simple)
    if simple.reactions:
      m.analyze(simple.reactions, error_details=False)
      results[file]["LP_ERROR"]
    else:
      

In [26]:
simple = SimpleSBML()
simple.initialize(os.path.join(data_dir, files[3]))
m = MESGraph(simple)
m.analyze(simple.reactions, error_details=False)
print(m.nodes)

[{s1}, {s16}, {s28}, {s27}, {s121}, {s123}, {s46}, {s129}, {s75}, {s159}, {s174}, {s176}, {s183}, {s173}, {s188}, {s252}, {s61}, {s259}, {s268}, {s266}, {s155}, {s267}, {s275}, {s101}, {s278}, {s164}, {s31}, {s288}, {s102}, {s107}, {s32}, {s33}, {s245}, {s179}, {s171}, {s170}, {s172}, {s195, s305}, {s292, s37}, {s286, s30}, {s239, s5}, {s232, s260, s270, s36}]


In [126]:
import networkx as nx
som_example = list(m.nodes)[-1]
PathNodesReactions = collections.namedtuple('PathNodesReactions',
    'node1 node2 reactions')
def getSOMPath(som, mole1, mole2):
  """
  Create an undirected graph between
  two molecules within a SOM
  and find the shortest path
  :param SOM som:
  :param str mole1:
  :param str mole2:
  :return PathNodesReactions result_data:
  """   
  molecule1 = simple.getMolecule(mole1).name
  moelcule2 = simple.getMolecule(mole2).name
  # construct undirected graph
  subg = nx.Graph()
  # here, every reaction is 1-1 reaction
  for reaction in list(som.reactions):
    print(reaction.makeIdentifier(is_include_kinetics=False))
    node1 = reaction.reactants[0].molecule.name
    node2 = reaction.products[0].molecule.name
    if subg.has_edge(node1, node2):
      reaction_label = subg.get_edge_data(node1, node2)[cn.REACTION]
      # if reaction.label is not already included in the attribute,
      if reaction.label not in set(reaction_label):
        reaction_label = reaction_label + [reaction.label]
    else:
      reaction_label = [reaction.label]    
    subg.add_edge(node1, node2, reaction=reaction_label)
  print(list(subg.edges))
  path = [p for p in nx.shortest_path(subg, 
                                      source=mole1, 
                                      target=mole2)]
  # if result has more than 1 element need a for loop
  print("We found the shortest path from " + mole1 + " to " + mole2)
  result_data = []
  for idx in range(len(path)-1):
    print(path[idx] + "=" + path[idx+1], end=" ")
    edge_reactions = subg.get_edge_data(path[idx], path[idx+1])[cn.REACTION]
    print("by reaction(s)", edge_reactions)
    result_data.append(PathNodesReactions(node1=path[idx], 
                                          node2=path[idx+1],
                                          reactions=edge_reactions))
  return result_data

In [131]:
som_path = getSOMPath(som_example, 's260', 's270')
type(som_path)

r58: s36 -> s232
re65: s260 -> s232
re64: s270 -> s232
[('s36', 's232'), ('s232', 's260'), ('s232', 's270')]
We found the shortest path from s260 to s270
s260=s232 by reaction(s) ['re65']
s232=s270 by reaction(s) ['re64']


list

In [134]:
for pat in som_path:

  print(pat.node1 + " = " + pat.node2 + " by", end=" ")
  for r in pat.reactions:
    reaction = simple.getReaction(r)
    print(reaction.makeIdentifier(is_include_kinetics=False))

s260 = s232 by re65: s260 -> s232
s232 = s270 by re64: s270 -> s232


In [124]:
print(subg.edges)
print(subg.has_edge('s260', 's232'))
print(subg.has_edge('s232', 's260'))

[('s36', 's232'), ('s232', 's260'), ('s232', 's270')]
True
True


In [87]:
som = som_example
mole1 = 's260'
mole2 = 's270'
molecule1 = simple.getMolecule(mole1).name
moelcule2 = simple.getMolecule(mole2).name
# construct undirected graph
subg = nx.Graph()
# subg.add_nodes_from(som.molecules)
# here, every reaction is 1-1 reaction
for reaction in list(som.reactions):
  print(reaction.makeIdentifier(is_include_kinetics=False))
  node1 = reaction.reactants[0].molecule.name
  node2 = reaction.products[0].molecule.name
  if subg.has_edge(node1, node2):
    reaction_label = subg.get_edge_data(node1, node2)[cn.REACTION]
    # if reaction.label is not already included in the attribute,
    if reaction.label not in set(reaction_label):
      reaction_label = reaction_label + [reaction.label]
  else:
    reaction_label = [reaction.label]    
  subg.add_edge(node1, node2, reaction=reaction_label)


r58: s36 -> s232
re65: s260 -> s232
re64: s270 -> s232


In [11]:
import tesbml
for error_path in error_paths:
  document = tesbml.readSBML(error_path)
  model = document.getModel()
  #pm.print_model(model)
  try:
    simple.initialize(error_path)
  except:
    print(error_path)
    print("showed error")

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000596_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/MODEL0568648427_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000410_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000081_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000075_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000094_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000353_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000627_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/MODEL0072364382_url.xml
showed error


In [7]:
#pm.print_model(model)
for rct in model.getReaction(0).getListOfReactants():
  print(rct.getSpecies())
  print("similar, ", rct.species)
  print(rct.stoichiometry)
  print("similar, ", rct.getStoichiometry())
print("rct done! now pdt")
for pdt in model.getReaction(0).getListOfProducts():
  print(pdt.getSpecies())
  print(pdt.stoichiometry)
  print("similar, ", pdt.getStoichiometry())
pm.print_model(model)

model.getReaction(3).getId()
[rct.stoichiometry for r in model.getReaction(3).getListOfReactants() \
if rct.species != "haha"]
#r = Reaction(model.getReaction(3))
#r.category

rct done! now pdt
s51
1.0
similar,  1.0
<Model MODEL1508180000 "Philipson2015 - Innate immune response modulated by NLRX1">
MyD88_a_HP:  -> s51;
MyD88_d: s51 -> ;
TRAF_d: s7 -> ;
NFkB_a1:  -> s4;
NFkB_d: s4 -> ;
CytoL_a:  -> s34;
CytoL_d: s34 -> ;
NLRX1_d: s49 -> ;
MyD88_a:  -> s51;
HP_a:  -> s18;
HP_CytoL: s18 -> ;
NLRX1_TF:  -> s49;
NFkB_a2:  -> s4;
RIG_a_HP:  -> RIG;
RIG_d: RIG -> ;
MAVS_a:  -> MAVS;
MAVS_d: MAVS -> ;
IRF_a1:  -> IRF;
IRF_a2:  -> IRF;
IRF_a3:  -> IRF;
IRF_d: IRF -> ;
IFN_a:  -> IFN;
IFN_d: IFN -> ;
NOD_a:  -> NOD1;
NOD_d: NOD1 -> ;
HP_IFN: s18 -> ;
TRAF_a:  -> s7;
TFa_a:  -> TFa;
TFa_d: TFa -> ;
TFi_a:  -> TFi;
TFi_d: TFi -> ;
CytoE_a:  -> CytoE;
CytoE_d: CytoE -> ;
X_a:  -> X;
X_d: X -> ;
HP_CytoE: s18 -> ;


[]

In [10]:
model.getLgetListOfReactants()

AttributeError: getListOfReactants

In [56]:
REACTION_1_1 = "reaction_1_1"
REACTION_n_1 = "reaction_n_1"
REACTION_1_n = "reaction_1_n"
REACTION_n_n = "reaction_n_n"
REACTION_BOUNDARY = "reaction_boundary"
ReactionCategory = collections.namedtuple('ReactionCategory',
    'category predicate')
REACTION_CATEGORIES = [
    ReactionCategory(category=REACTION_1_1,
        predicate=lambda x,y,z,w: (x==1) and (y==1) and (z==w)),
    ReactionCategory(category=REACTION_1_n,
        predicate=lambda x,y,z,w: ((x==1) and (y>1) and (z==1.00)) or ((x==1) and (y==1) and (z<w))),
    ReactionCategory(category=REACTION_n_1,
        predicate=lambda x,y,z,w: ((x>1) and (y==1) and (w==1.00)) or ((x==1) and (y==1) and (z>w))),
    ReactionCategory(category=REACTION_n_n,
        predicate=lambda x,y,z,w: ((x>1) and (y>1)) or ((x==1) and (y>1) and (z!=1.00)) or ((x>1) and (y==1) and (w!=1.00))),
    ReactionCategory(category=REACTION_BOUNDARY,
        predicate=lambda x,y,z,w: (x==0) or (y==0) or (z==0) or (w==0)),
    ]

In [57]:
for reaction in model.getListOfReactions():
  num_rct = len([r.species for r in reaction.getListOfReactants()])
  num_pdt = len([p.species for p in reaction.getListOfProducts()])
  stoi_rct = sum([r.stoichiometry for r in reaction.getListOfReactants()])
  stoi_pdt = sum([p.stoichiometry for p in reaction.getListOfProducts()])
  print("x =", num_rct, ";y =", num_pdt, ";z =", stoi_rct, ";w =", stoi_pdt)
  for reaction_category in REACTION_CATEGORIES:
    if reaction_category.predicate(num_rct, num_pdt, 
                                 stoi_rct, stoi_pdt):
      print(reaction_category.category)

x = 3 ;y = 1 ;z = 4.0 ;w = 2.0
reaction_n_n
x = 3 ;y = 1 ;z = 4.0 ;w = 1.5
reaction_n_n
x = 1 ;y = 1 ;z = 1.0 ;w = 1.0
reaction_1_1
x = 1 ;y = 1 ;z = 1.0 ;w = 1.0
reaction_1_1


In [49]:
pm.print_model(model)

<Model "Arnold2011_Damour2007_RuBisCO-CalvinCycle">
PGA_prod_Vc: RuBP + CO2 + NADPH -> PGA;
PGA_prod_Vo: RuBP + O2 + NADPH -> PGA;
PGA_cons: PGA -> RuBP;
NADPH_prod: NADP -> NADPH;


In [51]:
x = 3 ;y = 1 ;z = 4.0 ;w = 2.0
((x>1) and (y==1) and (w!=1.00))

True

In [53]:
((x>1) and (y>1)) or ((x==1) and (y>1) and (z!=1.00)) or ((x>1) and (y==1) and (w!=1.00))

True